# VAN-Detection Playground
A notebook for testing different configurations of VAN based Object Detectors.

## Setup

In [32]:
!nvidia-smi

Wed Nov  9 20:49:55 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.85.02    Driver Version: 510.85.02    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA TITAN RTX    Off  | 00000000:01:00.0 Off |                  N/A |
|  0%   36C    P8    13W / 280W |     87MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA TITAN RTX    Off  | 00000000:4D:00.0 Off |                  N/A |
|  0%   

In [1]:
!pwd

/home/nils/VAN-Detection


In [ ]:
# !echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
# !curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
# !apt -qq update
# !apt -qq install gcsfuse

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2537  100  2537    0     0  76878      0 --:--:-- --:--:-- --:--:-- 79281
OK
W: GPG error: https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease: The following signatures couldn't be verified because the public key is not available: NO_PUBKEY A4B469963BF863CC
E: The repository 'https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease' is no longer signed.
The following package was automatically installed and is no longer required:
  libnvidia-common-470
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 newly installed, 0 to remove and 302 not upgraded.
Need to get 12.1 MB of archives.
After this operation, 27.6 MB of additional disk space will be used.
Selecting previously unselected package gcsfuse.
(Reading da

In [ ]:
# !mkdir -p googleBucketFolder
# !gcsfuse --implicit-dirs colab-test-storage googleBucketFolder

2022/10/25 08:43:17.397120 Start gcsfuse/0.41.7 (Go version go1.18.4) for app "" using mount point: /content/googleBucketFolder
2022/10/25 08:43:17.408210 Opening GCS connection...
^C


In [3]:
!wget http://images.cocodataset.org/zips/train2017.zip -O datasets/coco_train2017.zip
!wget http://images.cocodataset.org/zips/val2017.zip -O datasets/coco_val2017.zip
!wget http://images.cocodataset.org/zips/test2017.zip -O datasets/coco_test2017.zip

--2022-10-31 16:15:23--  http://images.cocodataset.org/zips/train2017.zip
Resolving www-cache.intern.mi.hs-rm.de (www-cache.intern.mi.hs-rm.de)... 172.26.47.113
Connecting to www-cache.intern.mi.hs-rm.de (www-cache.intern.mi.hs-rm.de)|172.26.47.113|:8080... connected.
Proxy request sent, awaiting response... 200 OK
Length: 19336861798 (18G) [application/zip]
Saving to: ‘datasets/coco_train2017.zip’

datasets/coco_train 100%[===================>]  18,01G  31,1MB/s    in 11m 23s 

2022-10-31 16:26:51 (27,0 MB/s) - ‘datasets/coco_train2017.zip’ saved [19336861798/19336861798]

--2022-10-31 16:26:51--  http://images.cocodataset.org/zips/val2017.zip
Resolving www-cache.intern.mi.hs-rm.de (www-cache.intern.mi.hs-rm.de)... 172.26.47.113
Connecting to www-cache.intern.mi.hs-rm.de (www-cache.intern.mi.hs-rm.de)|172.26.47.113|:8080... connected.
Proxy request sent, awaiting response... 200 OK
Length: 815585330 (778M) [application/zip]
Saving to: ‘datasets/coco_val2017.zip’

datasets/coco_val20 1

In [ ]:
# This doesn't work with custom VMs
# Why did I ever expect anything else?
# Google doesn't seem to want to make you use their services in conjunction 
# how silly would that be?

# from google.colab import auth
# auth.authenticate_user()

KeyboardInterrupt: ignored

In [ ]:
# !pip install virtualenv
# !virtualenv van-experimentation
%cd /content/
!source van-experimentation/bin/activate

/content


In [9]:
!pip uninstall mmcv-full mmdet
!pip install openmim
!mim install mmcv-full==1.6.2 mmdet=2.25.2
# !pip install wandb timm pycocotools mmcv-full==1.6.2 mmdet==2.25.2
# !wandb login

  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [6 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/tmp/pip-install-rtmmwjg4/mim_a6acec9a65804dbdb9a1ab337af281f5/setup.py", line 43, in <module>
          import ConfigParser
      ModuleNotFoundError: No module named 'ConfigParser'
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
/bin/bash: line 1: mim: command not found


In [3]:
# Reinstall torch to compatible with our cuda version
!pip3 uninstall -y torch torchvision 
!pip3 install torch torchvision==0.13.0 --extra-index-url https://download.pytorch.org/whl/cu116

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 22.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 GB 595.7 kB/s eta 0:00:0000:010:02m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 35.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 25.0 MB/s eta 0:00:00:00:01


In [3]:
!pip install timm wandb

  Using cached timm-0.6.11-py3-none-any.whl (548 kB)
  Using cached wandb-0.13.4-py2.py3-none-any.whl (1.9 MB)
  Using cached huggingface_hub-0.10.1-py3-none-any.whl (163 kB)
  Using cached GitPython-3.1.29-py3-none-any.whl (182 kB)


In [1]:
import torch
print(f'CUDA available: {torch.cuda.is_available()}')
print(f'PyTorch version: {torch.__version__}')

CUDA available: True
PyTorch version: 1.12.0+cu116


/home/nils/miniconda3/envs/detection/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Training

In [17]:
#@title Config   { run: "auto" }
wandb_mode = 'online' #@param = ['online', 'offline', 'dryrun']

use_ref = False #@param = {type: 'boolean'}
folder = 'det_fpn' if not use_ref else 'refs'
test_config = 'fcos_van_b0_fpn_coco.py' #@param = ['fcos_van_b0_fpn_coco.py', 'fcos_van_b0_frozen_fpn_coco.py', 'fcos_van_b0_starrelu_fpn_coco.py', 'fcos_van_b2_fpn_coco.py', 'fcos_van_b2_frozen_fpn_coco.py', 'fcos_van_b2_StarReLU_fpn_dyhead_coco.py', 'fcos_van_b2_frozen_fpn_dyhead_coco.py', 'atss_van_b2_fpn_dyhead_coco.py']
ref_config = 'fcos_r50_caffe_fpn_gn-head_1x_coco.py' #@param = ['atss_r50_fpn_dyhead_1x_coco.py', 'fcos_r50_caffe_fpn_gn-head_1x_coco.py']

config = ref_config if use_ref else test_config

print(f'using {folder}/{config}')

from os import environ
environ['WANDB_MODE'] = wandb_mode

# # BP Mode
# environ['LOCAL_RANK'] = '3'

using det_fpn/fcos_van_b0_fpn_coco.py


In [1]:
from mmcv.runner import get_dist_info, init_dist

get_dist_info()

/home/nils/miniconda3/envs/detection/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(0, 1)

In [10]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	modified:   configs/_base_/datasets/coco_detection.py
	modified:   configs/det_fpn/fcos_van_b0_fpn_coco.py
	modified:   configs/det_fpn/fcos_van_b0_frozen_fpn_coco.py
	modified:   configs/det_fpn/fcos_van_b0_starrrelu_fpn_coco.py
	modified:   configs/det_fpn/fcos_van_b2_fpn_coco.py

Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   configs/_base_/datasets/coco_detection.py
	modified:   configs/_base_/default_runtime.py
	modified:   configs/_base_/schedules/schedule_1x.py
	modified:   configs/_base_/schedules/schedule_40k.py
	modified:   configs/det_fpn/fcos_van_b0_fpn_coco.py
	modified:   configs/det_fpn/fcos_van_b2_StarReLU_fpn_dyhead_coco.py
	deleted:    configs/refs/atss_r50_fpn_dyhead_1x_coco.py.py
	modified:   train

In [ ]:
# %cd /content/VAN-Detection/
# !git pull
!torchrun train.py configs/{folder}/{config} --gpus=3 --local_rank=3

/home/nils/VAN-Detection/train.py:142: UserWarning: `--gpus` is deprecated because we only support single GPU mode in non-distributed training. Use `gpus=1` now.
  warnings.warn('`--gpus` is deprecated because we only support '
/home/nils/miniconda3/envs/detection/lib/python3.10/site-packages/mmdet/utils/setup_env.py:38: UserWarning: Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  warnings.warn(
/home/nils/miniconda3/envs/detection/lib/python3.10/site-packages/mmdet/utils/setup_env.py:48: UserWarning: Setting MKL_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  warnings.warn(
2022-11-04 12:31:30,768 - mmdet - INFO - Environment info:
---------------------------------------------

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



2022-11-05 10:18:29,152 - mmdet - INFO - Epoch [7][14050/29317]	lr: 4.972e-05, eta: 17:55:48, time: 0.395, data_time: 0.008, memory: 22273, loss_cls: 0.2374, loss_bbox: 0.3133, loss_centerness: 0.5944, loss: 1.1451
2022-11-05 10:18:48,668 - mmdet - INFO - Epoch [7][14100/29317]	lr: 4.971e-05, eta: 17:55:28, time: 0.390, data_time: 0.008, memory: 22273, loss_cls: 0.2364, loss_bbox: 0.2978, loss_centerness: 0.5931, loss: 1.1273
2022-11-05 10:19:08,223 - mmdet - INFO - Epoch [7][14150/29317]	lr: 4.969e-05, eta: 17:55:08, time: 0.391, data_time: 0.008, memory: 22273, loss_cls: 0.2236, loss_bbox: 0.2948, loss_centerness: 0.5916, loss: 1.1101
2022-11-05 10:19:27,791 - mmdet - INFO - Epoch [7][14200/29317]	lr: 4.968e-05, eta: 17:54:48, time: 0.391, data_time: 0.008, memory: 22273, loss_cls: 0.2364, loss_bbox: 0.2987, loss_centerness: 0.5907, loss: 1.1258
2022-11-05 10:19:47,729 - mmdet - INFO - Epoch [7][14250/29317]	lr: 4.967e-05, eta: 17:54:28, time: 0.399, data_time: 0.008, memory: 22273, 

In [ ]:
# Just for staging so it runs while I'm gone for a while
# This is for testing of StarReLU
%cd /content/VAN-Detection/
# config = 'fcos_van_b2_fpn_dyhead_coco.py'
!/bin/bash dist_train.sh configs/{folder}/{config} 1 --gpus=1

/content/VAN-Detection
Traceback (most recent call last):
  File "train.py", line 254, in <module>
    main()
  File "train.py", line 118, in main
    cfg = Config.fromfile(args.config)
  File "/usr/local/lib/python3.7/dist-packages/mmcv/utils/config.py", line 341, in fromfile
    use_predefined_variables)
  File "/usr/local/lib/python3.7/dist-packages/mmcv/utils/config.py", line 183, in _file2dict
    check_file_exist(filename)
  File "/usr/local/lib/python3.7/dist-packages/mmcv/utils/path.py", line 23, in check_file_exist
    raise FileNotFoundError(msg_tmpl.format(filename))
FileNotFoundError: file "/content/VAN-Detection/configs/det_fpn/fcos_van_b0_starrelu_fpn_coco.py" does not exist
ERROR:torch.distributed.elastic.multiprocessing.api:failed (exitcode: 1) local_rank: 0 (pid: 15316) of binary: /usr/bin/python3
Traceback (most recent call last):
  File "/usr/local/bin/torchrun", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.7/dist-packages/torch/distributed

In [ ]:
# Just for staging so it runs while I'm gone for a while
# Once more, but for bigger VAN without StarReLU
# This might fail due to OO(V)M
%cd /content/VAN-Detection/
config = 'fcos_van_b2_fpn_coco.py'
!/bin/bash dist_train.sh configs/{folder}/{config} 1 --gpus=1

/content/VAN-Detection
/usr/local/lib/python3.7/dist-packages/mmdet/utils/setup_env.py:39: UserWarning: Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  f'Setting OMP_NUM_THREADS environment variable for each process '
/usr/local/lib/python3.7/dist-packages/mmdet/utils/setup_env.py:49: UserWarning: Setting MKL_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  f'Setting MKL_NUM_THREADS environment variable for each process '
2022-10-28 08:09:23,951 - mmdet - INFO - Environment info:
------------------------------------------------------------
sys.platform: linux
Python: 3.7.12 (default, Jan 15 2022, 18:48:18) [GCC 7.5.0]
CUDA available: True
GPU 0: Tesla T4
CUDA_HOME: /usr/local/cu

In [54]:
# convert state dict where keys start with e.g. backbone.*
import torch
chkpt = torch.load('/home/nils/VAN-Detection/models/van-base_8xb128_in1k_20220501-6a4cc31b.pth')
torch.nn.modules.utils.consume_prefix_in_state_dict_if_present(chkpt['state_dict'],prefix='backbone.')
# print(chkpt['state_dict'].keys())
torch.save(chkpt, '/home/nils/VAN-Detection/models/van-base_8xb128_in1k_20220501-conv.pth')

odict_keys(['head.fc.weight', 'head.fc.bias', 'blocks1.0.attn.proj_1.bias', 'blocks1.0.attn.proj_1.weight', 'blocks1.0.attn.proj_2.bias', 'blocks1.0.attn.proj_2.weight', 'blocks1.0.attn.spatial_gating_unit.DW_D_conv.bias', 'blocks1.0.attn.spatial_gating_unit.DW_D_conv.weight', 'blocks1.0.attn.spatial_gating_unit.DW_conv.bias', 'blocks1.0.attn.spatial_gating_unit.DW_conv.weight', 'blocks1.0.attn.spatial_gating_unit.conv1.bias', 'blocks1.0.attn.spatial_gating_unit.conv1.weight', 'blocks1.0.layer_scale_1', 'blocks1.0.layer_scale_2', 'blocks1.0.mlp.dwconv.bias', 'blocks1.0.mlp.dwconv.weight', 'blocks1.0.mlp.fc1.bias', 'blocks1.0.mlp.fc1.weight', 'blocks1.0.mlp.fc2.bias', 'blocks1.0.mlp.fc2.weight', 'blocks1.0.norm1.bias', 'blocks1.0.norm1.num_batches_tracked', 'blocks1.0.norm1.running_mean', 'blocks1.0.norm1.running_var', 'blocks1.0.norm1.weight', 'blocks1.0.norm2.bias', 'blocks1.0.norm2.num_batches_tracked', 'blocks1.0.norm2.running_mean', 'blocks1.0.norm2.running_var', 'blocks1.0.norm2.w

In [ ]:
# Just for staging so it runs while I'm gone for a while
# Once more, but for bigger VAN without StarReLU
# This might fail due to OO(V)M
%env NCCL_P2P_DISABLE="1" 
%env NCCL_IB_DISABLE="1" 
%env MKL_NUM_THREADS=8 
%env OMP_NUM_THREADS=8 
%env CUDA_VISIBLE_DEVICES=2 
%env WANDB_MODE=online 
%cd /home/nils/VAN-Detection/mmdetection/
!/bin/bash tools/dist_train.sh ../configs/det_fpn/fcos_van_b2_fpn_1x_coco_adam_newvan.py 1


env: NCCL_P2P_DISABLE="1"
env: NCCL_IB_DISABLE="1"
env: MKL_NUM_THREADS=8
env: OMP_NUM_THREADS=8
env: CUDA_VISIBLE_DEVICES=2
env: WANDB_MODE=online
/home/nils/VAN-Detection/mmdetection
/home/nils/miniconda3/envs/detection/lib/python3.10/site-packages/torch/distributed/launch.py:178: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use_env is set by default in torchrun.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  warnings.warn(
2022-11-15 18:57:41,220 - mmdet - INFO - Environment info:
------------------------------------------------------------
sys.platform: linux
Python: 3.10.0 (default, Mar  3 2022, 09:58:08) [GCC 7.5.0]
CUDA available: True
GPU 0: NVIDIA TITAN RTX
CUDA_HOME: /usr/local/cuda-11.6
NVCC: Cuda compilation tools, release 11.

In [ ]:
# Just for staging so it runs while I'm gone for a while
# Once more, but for bigger VAN without StarReLU
# This might fail due to OO(V)M
%env NCCL_P2P_DISABLE="1" 
%env NCCL_IB_DISABLE="1" 
%env MKL_NUM_THREADS=8 
%env OMP_NUM_THREADS=8 
%env CUDA_VISIBLE_DEVICES=0
%env WANDB_MODE=online 
!/bin/bash dist_train.sh configs/det_fpn/fcos_van_b0_fpn_dcn_1x_coco.py 1


env: NCCL_P2P_DISABLE="1"
env: NCCL_IB_DISABLE="1"
env: MKL_NUM_THREADS=8
env: OMP_NUM_THREADS=8
env: CUDA_VISIBLE_DEVICES=0
env: WANDB_MODE=online
/home/nils/miniconda3/envs/detection/lib/python3.10/site-packages/mmdet/utils/setup_env.py:17: UserWarning: Multi-processing start method `fork` is different from the previous setting `spawn`.It will be force set to `fork`. You can change this behavior by changing `mp_start_method` in your config.
  warnings.warn(
2022-11-08 11:12:20,788 - mmdet - INFO - Environment info:
------------------------------------------------------------
sys.platform: linux
Python: 3.10.0 (default, Mar  3 2022, 09:58:08) [GCC 7.5.0]
CUDA available: True
GPU 0: NVIDIA TITAN RTX
CUDA_HOME: /usr/local/cuda-11.6
NVCC: Cuda compilation tools, release 11.6, V11.6.55
GCC: gcc (Ubuntu 11.3.0-1ubuntu1~22.04) 11.3.0
PyTorch: 1.12.0+cu116
PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Prod

# Utils'n'Stuff

In [ ]:
from mmdet import models
import torch
print(dir(models))

self = models.ResNet(depth=50)
self.eval()
inputs = torch.rand(1,3,32,32)
level_outputs = self.forward(inputs)
for level_out in level_outputs:
  # Format (1, dim, idk, idk)
  print(tuple(level_out.shape))
# print(models.__file__)

['ATSS', 'ATSSHead', 'Accuracy', 'AnchorFreeHead', 'AnchorHead', 'AssociativeEmbeddingLoss', 'AutoAssign', 'AutoAssignHead', 'BACKBONES', 'BBoxHead', 'BFP', 'BalancedL1Loss', 'BaseDetector', 'BasePanopticFusionHead', 'BaseRoIExtractor', 'BaseRoIHead', 'BoundedIoULoss', 'CIoULoss', 'CSPDarknet', 'CTResNetNeck', 'CascadeRCNN', 'CascadeRPNHead', 'CascadeRoIHead', 'CenterNet', 'CenterNetHead', 'CentripetalHead', 'ChannelMapper', 'CoarseMaskHead', 'ConvFCBBoxHead', 'CornerHead', 'CornerNet', 'CrossEntropyLoss', 'DDOD', 'DDODHead', 'DETECTORS', 'DETR', 'DETRHead', 'DIIHead', 'DIoULoss', 'Darknet', 'DecoupledSOLOHead', 'DecoupledSOLOLightHead', 'DeformableDETR', 'DeformableDETRHead', 'DetectoRS_ResNeXt', 'DetectoRS_ResNet', 'DiceLoss', 'DilatedEncoder', 'DistributionFocalLoss', 'DoubleConvFCBBoxHead', 'DoubleHeadRoIHead', 'DropBlock', 'DyHead', 'DynamicRoIHead', 'EfficientNet', 'EmbeddingRPNHead', 'FCNMaskHead', 'FCOS', 'FCOSHead', 'FOVEA', 'FPG', 'FPN', 'FPN_CARAFE', 'FSAF', 'FSAFHead', 'Fas

In [ ]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   configs/_base_/default_runtime.py
	modified:   configs/det_fpn/fcos_van_b0_fpn_coco.py
	modified:   configs/det_fpn/fcos_van_b2_fpn_coco.py
	modified:   train.py

no changes added to commit (use "git add" and/or "git commit -a")


In [ ]:
!git add *
!git commit